In [251]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

from tqdm import tqdm_notebook
import lightgbm as lgb
from catboost import Pool, CatBoostClassifier

import warnings
warnings.filterwarnings("ignore")

import seaborn as sns



plt.style.use('seaborn')
sns.set(font_scale=1)

import gc

In [252]:
train = pd.read_csv("../input/cmu-africa-data-science-club-challenge/train (9).csv")
test = pd.read_csv("../input/cmu-africa-data-science-club-challenge/test (10).csv")
sample = pd.read_csv('../input/cmu-africa-data-science-club-challenge/sample_submission_fixed.csv')

In [253]:
id_=test["id"]
test.drop("id", axis=1, inplace=True)

In [254]:
cols=['REGION-CENTROID-COL', 'REGION-CENTROID-ROW', 'REGION-PIXEL-COUNT',
       'SHORT-LINE-DENSITY-5', 'SHORT-LINE-DENSITY-2', 'VEDGE-MEAN',
       'VEDGE-SD', 'HEDGE-MEAN', 'HEDGE-SD', 'INTENSITY-MEAN', 'RAWRED-MEAN',
       'RAWBLUE-MEAN', 'RAWGREEN-MEAN', 'EXRED-MEAN', 'EXBLUE-MEAN',
       'EXGREEN-MEAN', 'VALUE-MEAN', 'SATURATION-MEAN', 'HUE-MEAN']

In [ ]:
df_train["mean"]=df_train.mean(axis=1)
df_train["count"]=df_train.count(axis=1)
df_train["std"]=df_train.std(axis=1)
df_train["min"]=df_train.min(axis=1)
df_train["max"]=df_train.max(axis=1)
df_train["kurt"]=df_train.kurt(axis=1)
df_train["skew"]=df_train.skew(axis=1)
df_train["q1"]=df_train.quantile(q=0.1,axis=1)
df_train["q2"]=df_train.quantile(q=0.2,axis=1)
df_train["q25"]=df_train.quantile(q=0.25,axis=1)
df_train["q3"]=df_train.quantile(q=0.3,axis=1)
df_train["q4"]=df_train.quantile(q=0.4,axis=1)
df_train["q5"]=df_train.quantile(q=0.5,axis=1)
df_train["q6"]=df_train.quantile(q=0.6,axis=1)
df_train["q7"]=df_train.quantile(q=0.7,axis=1)
df_train["q75"]=df_train.quantile(q=0.75,axis=1)
df_train["q8"]=df_train.quantile(q=0.8,axis=1)
df_train["q9"]=df_train.quantile(q=0.9,axis=1)

In [258]:
# y_train = train[['CLASS_0','CLASS_1', 'CLASS_2', 'CLASS_3', 'CLASS_4', 'CLASS_5', 'CLASS_6']]
y_train = train[['CLASS']]
train_df=train.drop(['CLASS'], axis=1)

In [259]:
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=5168)
# oof = df_train[['ID_code', 'target']]
# oof['predict'] = 0
# predictions = test[['ID_code']]
# feature_importance_df = pd.DataFrame()
# val_aucs = []
for fold, (trn_idx, val_idx) in enumerate(skf.split(train_df, y_train)):
    X_train, y_train = train_df.iloc[trn_idx][cols], train.iloc[trn_idx]['CLASS']
    X_valid, y_valid = train_df.iloc[val_idx][cols], train.iloc[val_idx]['CLASS']
    break
    
    
    
clf = CatBoostClassifier(loss_function = "MultiClass", eval_metric = "AUC",random_seed=123,use_best_model=True,
                          learning_rate=0.01,  iterations=25000,verbose=100,
                           bootstrap_type= "Poisson", 
                           task_type="GPU", 
#                              l2_leaf_reg= 16.5056753964314982, depth= 3.0,
#                              fold_len_multiplier= 2.9772639036842174, 
#                              scale_pos_weight= 3.542962442406767, 
#                              fold_permutation_block_size=16.0, subsample= 0.46893530376570957
#                              fold_len_multiplier=3.2685541035861747, 
#                              scale_pos_weight= 2.6496926337120916, 
#                              fold_permutation_block_size= 6.0, 
                          )
print("Model training")
clf.fit(X_train, y_train,  eval_set=(X_valid, y_valid), early_stopping_rounds=2000,verbose=100)

Model training
0:	test: 0.9521429	best: 0.9521429 (0)	total: 14.6ms	remaining: 6m 3s


Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


100:	test: 0.9980952	best: 0.9980952 (32)	total: 1.16s	remaining: 4m 45s
200:	test: 0.9980952	best: 0.9980952 (32)	total: 2.3s	remaining: 4m 43s
300:	test: 0.9985714	best: 0.9985714 (268)	total: 3.45s	remaining: 4m 42s
400:	test: 0.9985714	best: 0.9985714 (268)	total: 4.57s	remaining: 4m 40s
500:	test: 0.9985714	best: 0.9985714 (268)	total: 5.72s	remaining: 4m 39s
600:	test: 0.9985714	best: 0.9985714 (268)	total: 6.92s	remaining: 4m 41s
700:	test: 0.9985714	best: 0.9985714 (268)	total: 8.18s	remaining: 4m 43s
800:	test: 0.9985714	best: 0.9985714 (268)	total: 9.37s	remaining: 4m 43s
900:	test: 0.9985714	best: 0.9985714 (268)	total: 11s	remaining: 4m 54s
1000:	test: 0.9985714	best: 0.9985714 (268)	total: 12.3s	remaining: 4m 54s
1100:	test: 0.9985714	best: 0.9985714 (268)	total: 14.1s	remaining: 5m 5s
1200:	test: 0.9985714	best: 0.9985714 (268)	total: 15.2s	remaining: 5m 1s
1300:	test: 0.9985714	best: 0.9985714 (268)	total: 16.4s	remaining: 4m 58s
1400:	test: 0.9985714	best: 0.9985714 (26

In [260]:
predict = clf.predict_proba(test[cols])

In [261]:
sub=pd.DataFrame({"id":id_,"CLASS_0":predict[:,0] ,"CLASS_1":predict[:,1],"CLASS_2":predict[:,2],"CLASS_3":predict[:,3],"CLASS_4":predict[:,4],"CLASS_5":predict[:,5],"CLASS_6":predict[:,6]})

In [262]:
sub

,id,CLASS_0,CLASS_1,CLASS_2,CLASS_3,CLASS_4,CLASS_5,CLASS_6
0,0,0.082984,0.027162,0.821969,0.012780,0.012704,0.021158,0.021244
1,1,0.009226,0.009936,0.015186,0.010738,0.856938,0.021475,0.076501
2,2,0.005130,0.005578,0.002505,0.964470,0.004582,0.002419,0.015315
3,3,0.003454,0.002631,0.979616,0.002454,0.002576,0.003045,0.006223
4,4,0.004873,0.005347,0.002751,0.973188,0.003626,0.002408,0.007807
...,...,...,...,...,...,...,...,...
2095,2095,0.006693,0.002621,0.960720,0.002668,0.003397,0.003652,0.020250
2096,2096,0.196474,0.026758,0.029670,0.064430,0.030096,0.019277,0.633295
2097,2097,0.015212,0.006111,0.005440,0.929372,0.007295,0.005300,0.031270
2098,2098,0.001317,0.000977,0.001771,0.001092,0.002170,0.989384,0.003289


In [263]:
from IPython.display import FileLink
def create_submission(submission_file, submission_name):
    submission_file.to_csv(submission_name+".csv",index=False)
    return FileLink(submission_name+".csv")


In [264]:
create_submission(sub, "sol")

/kaggle/working/sol.csv